# Dzień 1 - Warsztaty Praktyczne

## Cele warsztatów:
- Zastosowanie poznanych technik w praktycznych projektach
- Tokenizacja i analiza danych tekstowych
- Eksploracja i przygotowanie danych
- Budowa prostego modelu klasyfikacji tekstu

## Projekty do realizacji:
1. Analiza sentymentu recenzji
2. Klasyfikacja tekstów (spam detection)
3. Eksploracja korpusu tekstowego
4. Mini-projekt: Budowa klasyfikatora

In [ ]:
# Import wszystkich potrzebnych bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud

import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Konfiguracja
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pobierz zasoby NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("✅ Biblioteki załadowane!")

## Projekt 1: Analiza Sentymentu Recenzji

### Cel:
Przeanalizuj recenzje produktów i określ ich sentyment (pozytywny/negatywny).

In [ ]:
# Stwórzmy zbiór przykładowych recenzji
recenzje = [
    ("Świetny produkt! Bardzo jestem zadowolony. Polecam!", "pozytywny"),
    ("Okropne. Nie działa tak jak powinno. Zwracam.", "negatywny"),
    ("Całkiem niezły, ale cena mogłaby być niższa.", "neutralny"),
    ("Najlepszy zakup w tym roku! Fantastyczna jakość!", "pozytywny"),
    ("Totalna porażka. Strata pieniędzy.", "negatywny"),
    ("Produkt jest OK, nic specjalnego.", "neutralny"),
    ("Rewelacja! Dokładnie to czego szukałem!", "pozytywny"),
    ("Nie polecam. Słaba jakość wykonania.", "negatywny"),
    ("Za tę cenę spodziewałem się więcej.", "negatywny"),
    ("Solidny produkt. Spełnia oczekiwania.", "pozytywny"),
]

# Utwórz DataFrame
df = pd.DataFrame(recenzje, columns=['tekst', 'sentyment'])

print("=== ZBIÓR DANYCH ===")
print(df)
print(f"\nRozkład klas:")
print(df['sentyment'].value_counts())

In [ ]:
# Analiza eksploracyjna
print("\n=== STATYSTYKI PODSTAWOWE ===")

# Długość tekstów
df['dlugosc'] = df['tekst'].apply(len)
df['liczba_slow'] = df['tekst'].apply(lambda x: len(x.split()))

print(df[['tekst', 'sentyment', 'dlugosc', 'liczba_slow']].head())

# Statystyki
print("\nŚrednia długość tekstu:", df['dlugosc'].mean())
print("Średnia liczba słów:", df['liczba_slow'].mean())

In [ ]:
# Wizualizacja rozkładu sentymentu
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
df['sentyment'].value_counts().plot(kind='bar')
plt.title('Rozkład sentymentu')
plt.xlabel('Sentyment')
plt.ylabel('Liczba recenzji')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
df.groupby('sentyment')['liczba_slow'].mean().plot(kind='bar', color='coral')
plt.title('Średnia liczba słów według sentymentu')
plt.xlabel('Sentyment')
plt.ylabel('Średnia liczba słów')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Analiza najczęstszych słów
from collections import Counter

# Tokenizacja i liczenie słów
wszystkie_slowa = []
stop_words = set(stopwords.words('polish'))

for tekst in df['tekst']:
    tokens = word_tokenize(tekst.lower(), language='polish')
    tokens = [t for t in tokens if t.isalpha() and t not in stop_words]
    wszystkie_slowa.extend(tokens)

# Najczęstsze słowa
najczestsze = Counter(wszystkie_slowa).most_common(15)

print("\n=== 15 NAJCZĘSTSZYCH SŁÓW ===")
for slowo, liczba in najczestsze:
    print(f"{slowo:20} : {liczba}")

In [ ]:
# WordCloud - chmura słów
try:
    tekst_caly = ' '.join(wszystkie_slowa)
    
    wordcloud = WordCloud(width=800, height=400, 
                          background_color='white',
                          colormap='viridis').generate(tekst_caly)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Chmura słów z recenzji', fontsize=16)
    plt.show()
except Exception as e:
    print(f"Błąd generowania chmury słów: {e}")
    print("Zainstaluj: pip install wordcloud")

## Projekt 2: Klasyfikacja Spam vs Ham (SMS)

### Cel:
Zbuduj prosty klasyfikator do rozróżniania spamu od prawidłowych wiadomości.

In [ ]:
# Przykładowy dataset SMS
sms_data = [
    ("Wygrałeś 1000 zł! Kliknij link aby odebrać nagrodę!", "spam"),
    ("Cześć, spotkamy się jutro o 18?", "ham"),
    ("GRATULACJE! Zostałeś wybrany do otrzymania iPhone'a!", "spam"),
    ("Pamiętaj o spotkaniu z klientem w czwartek", "ham"),
    ("Kup teraz! Wielka promocja tylko dziś! -90%", "spam"),
    ("Dzięki za wczorajsze spotkanie. Pozdrawiam", "ham"),
    ("Pilne! Twoje konto zostało zablokowane. Kliknij tutaj", "spam"),
    ("Mama dzwoniła, oddzwoń jak będziesz miał czas", "ham"),
    ("DARMOWE PIENIĄDZE! Nie czekaj, zarejestruj się!", "spam"),
    ("Kupiłem mleko, wracam za godzinę", "ham"),
    ("Ostatnia szansa! Oferta wygasa za 2 godziny!!!", "spam"),
    ("Spotkanie przeniesione na piątek o 14:00", "ham"),
]

df_sms = pd.DataFrame(sms_data, columns=['wiadomosc', 'kategoria'])

print("=== DATASET SMS ===")
print(df_sms)
print(f"\nRozkład klas:")
print(df_sms['kategoria'].value_counts())

In [ ]:
# Preprocessing - pipeline
def preprocess_text(text):
    """
    Przetwarzanie tekstu przed klasyfikacją.
    """
    # Lowercase
    text = text.lower()
    
    # Tokenizacja
    tokens = word_tokenize(text, language='polish')
    
    # Usuń znaki specjalne i liczby
    tokens = [t for t in tokens if t.isalpha()]
    
    # Usuń stop words
    stop_words = set(stopwords.words('polish'))
    tokens = [t for t in tokens if t not in stop_words]
    
    return ' '.join(tokens)

# Zastosuj preprocessing
df_sms['wiadomosc_przetworzona'] = df_sms['wiadomosc'].apply(preprocess_text)

print("=== PRZYKŁADY PRZETWARZANIA ===")
for idx in [0, 1, 4]:
    print(f"\nOryginał: {df_sms.iloc[idx]['wiadomosc']}")
    print(f"Przetworzone: {df_sms.iloc[idx]['wiadomosc_przetworzona']}")

In [ ]:
# Feature Engineering - TF-IDF
vectorizer = TfidfVectorizer(max_features=50)

X = vectorizer.fit_transform(df_sms['wiadomosc_przetworzona'])
y = df_sms['kategoria']

print("=== FEATURE ENGINEERING ===")
print(f"Shape macierzy cech: {X.shape}")
print(f"\nNajważniejsze cechy (słowa):")
print(vectorizer.get_feature_names_out()[:20])

In [ ]:
# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("=== PODZIAŁ DANYCH ===")
print(f"Zbiór treningowy: {X_train.shape[0]} próbek")
print(f"Zbiór testowy: {X_test.shape[0]} próbek")

In [ ]:
# Trening modelu - Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Predykcja
y_pred = model.predict(X_test)

print("=== WYNIKI MODELU ===")
print(f"\nDokładność: {accuracy_score(y_test, y_pred):.2%}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred))

In [ ]:
# Test modelu na nowych danych
nowe_wiadomosci = [
    "Wygrałeś milion złotych! Kliknij teraz!",
    "Spotkamy się w kawiarni o 16?",
    "PROMOCJA! Kup teraz ze zniżką 99%",
]

print("\n=== TEST NA NOWYCH DANYCH ===")
for wiadomosc in nowe_wiadomosci:
    przetworzona = preprocess_text(wiadomosc)
    wektor = vectorizer.transform([przetworzona])
    predykcja = model.predict(wektor)[0]
    prawdopodobienstwo = model.predict_proba(wektor)[0]
    
    print(f"\nWiadomość: {wiadomosc}")
    print(f"Predykcja: {predykcja}")
    print(f"Prawdopodobieństwo: ham={prawdopodobienstwo[0]:.2%}, spam={prawdopodobienstwo[1]:.2%}")

## Projekt 3: Eksploracja korpusu tekstowego

### Cel:
Przeanalizuj większy zbiór tekstów i wyciągnij interesujące wnioski.

In [ ]:
# Utwórz większy korpus (symulacja artykułów)
artykuly = [
    {
        'tytul': 'Sztuczna inteligencja w medycynie',
        'tekst': 'Sztuczna inteligencja rewolucjonizuje medycynę. Algorytmy uczenia maszynowego pomagają w diagnostyce chorób. Systemy AI analizują obrazy medyczne z dokładnością przewyższającą ludzkich ekspertów.',
        'kategoria': 'technologia'
    },
    {
        'tytul': 'Nowe odkrycie archeologiczne',
        'tekst': 'Archeolodzy odkryli starożytne miasto. Znaleziska obejmują ceramikę, narzędzia i ruiny budynków. Odkrycie rzuca nowe światło na historię starożytnych cywilizacji.',
        'kategoria': 'historia'
    },
    {
        'tytul': 'Przyszłość transportu',
        'tekst': 'Pojazdy autonomiczne to przyszłość transportu. Samochody bez kierowcy będą bezpieczniejsze i bardziej efektywne. Technologia ta zmieni sposób w jaki się przemieszczamy.',
        'kategoria': 'technologia'
    },
    {
        'tytul': 'Zmiany klimatyczne',
        'tekst': 'Globalne ocieplenie wpływa na ekosystemy. Naukowcy ostrzegają przed konsekwencjami zmian klimatycznych. Konieczne są pilne działania aby chronić naszą planetę.',
        'kategoria': 'środowisko'
    },
]

df_artykuly = pd.DataFrame(artykuly)
print("=== KORPUS ARTYKUŁÓW ===")
print(df_artykuly[['tytul', 'kategoria']])

In [ ]:
# Analiza z użyciem spaCy
try:
    nlp = spacy.load("pl_core_news_sm")
except:
    print("⚠️ Zainstaluj model polski: python -m spacy download pl_core_news_sm")
    nlp = None

if nlp:
    print("\n=== ANALIZA LINGUISTYCZNA ===")
    
    for idx, row in df_artykuly.iterrows():
        doc = nlp(row['tekst'])
        
        print(f"\n{row['tytul']}")
        print("-" * 50)
        
        # Rzeczowniki
        rzeczowniki = [token.lemma_ for token in doc if token.pos_ == 'NOUN']
        print(f"Kluczowe rzeczowniki: {list(set(rzeczowniki))[:5]}")
        
        # Nazwane encje
        if doc.ents:
            encje = [(ent.text, ent.label_) for ent in doc.ents]
            print(f"Encje: {encje}")
        
        # Statystyki
        print(f"Liczba zdań: {len(list(doc.sents))}")
        print(f"Liczba tokenów: {len(doc)}")

## Ćwiczenie finalne: Mini-projekt

### Zadanie:
Stwórz kompletny pipeline klasyfikacji tekstu:
1. Załaduj lub stwórz dane
2. Wykonaj eksploracyjną analizę danych (EDA)
3. Przetwórz tekst (preprocessing)
4. Wyekstraktuj cechy (TF-IDF lub inne)
5. Wytrenuj model (wybierz algorytm)
6. Oceń model (metryki, confusion matrix)
7. Przetestuj na nowych danych

### Wskazówki:
- Używaj poznanych technik z całego dnia
- Eksperymentuj z różnymi parametrami
- Porównaj różne modele (Naive Bayes, Logistic Regression)
- Wizualizuj wyniki

In [ ]:
# TWOJA IMPLEMENTACJA
# Rozpocznij tutaj swoją pracę nad mini-projektem

# Przykładowa struktura:

# 1. Załaduj dane
# data = ...

# 2. EDA
# ...

# 3. Preprocessing
# ...

# 4. Feature extraction
# ...

# 5. Model training
# ...

# 6. Evaluation
# ...

# 7. Testing
# ...

print("🚀 Powodzenia w realizacji projektu!")

## Dodatkowe wyzwania (opcjonalne)

### Wyzwanie 1: Porównanie modeli
Porównaj wydajność różnych algorytmów:
- Naive Bayes
- Logistic Regression
- SVM
- Random Forest

### Wyzwanie 2: Feature engineering
Spróbuj różnych metod ekstrakcji cech:
- Bag of Words
- TF-IDF
- N-gramy (bigrams, trigrams)
- Word embeddings

### Wyzwanie 3: Optymalizacja
Użyj GridSearchCV do znalezienia najlepszych hiperparametrów.

### Wyzwanie 4: Analiza błędów
Przeanalizuj przypadki w których model się myli - co można poprawić?

## Podsumowanie Dnia 1

### Czego się nauczyliśmy:
✅ Podstawy NLP i jego zastosowań

✅ Narzędzia: NLTK, spaCy, Hugging Face, OpenAI

✅ Operacje: tokenizacja, lematyzacja, POS tagging

✅ Praktyczne projekty: analiza sentymentu, spam detection

✅ Pipeline ML: od danych do modelu

### Na jutro:
🚀 Nowoczesne modele NLP (Transformery)

🚀 BERT, GPT i fine-tuning

🚀 Generowanie i rozumienie tekstu

🚀 Zastosowania biznesowe (chatboty, automatyzacja)

---

**Dobra robota! 🎉 Do zobaczenia jutro!**